In [127]:
from textblob import TextBlob
import sys
import tweepy
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import nltk
import pycountry
import tweepy as tw

In [107]:
import re
import string
from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from langdetect import detect

ModuleNotFoundError: No module named 'langdetect'

In [108]:
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer

In [133]:
import spacy
nlp = spacy.load("en_core_web_sm")
from spacytextblob.spacytextblob import SpacyTextBlob
nlp.add_pipe('spacytextblob')

In [82]:
search_queries = ['"University of Chicago" innovation lang:en -is:retweet',
'UChicago innovation lang:en -is:retweet','"University of Chicago" research lang:en -is:retweet',
'UChicago research lang:en -is:retweet','UChicago SARS-CoV-2 lang:en -is:retweet',
'UChicago Therapeutics lang:en -is:retweet','UChicago diagnostics lang:en -is:retweet', 
'UChicago research tools lang:en -is:retweet','UChicago medical devices lang:en -is:retweet',
'UChicago materials lang:en -is:retweet','UChicago quantum sciences lang:en -is:retweet',
'UChicago hardware lang:en -is:retweet','UChicago software lang:en -is:retweet',
'UChicago cleantech lang:en -is:retweet','UChicago education lang:en -is:retweet',
'UChicago agricultural lang:en -is:retweet']



In [128]:
client = tweepy.Client(bearer_token='AAAAAAAAAAAAAAAAAAAAANRIlgEAAAAAK%2Bd8RT9ZcaOUTXPg2dksmEBaTE8%3D7CtgMOdxsysQYaqTE9AI0gwd4qHyVgpa3jSYSxIm8FzmAAOm1P')

In [129]:
search_queries = ['"University of Chicago" innovation lang:en -is:retweet',
'UChicago innovation lang:en -is:retweet','"University of Chicago" research lang:en -is:retweet',
'UChicago research lang:en -is:retweet',
'UChicago Therapeutics lang:en -is:retweet',
'UChicago materials lang:en -is:retweet','UChicago education lang:en -is:retweet']

#removed search queries without matches

In [156]:
#Sentiment Analysis

sent_labels=[]
sent_scores = []
total_pos=[]
total_neg=[]
tweet_list = []
keywords = []

for query in search_queries:
    tweets = client.search_recent_tweets(query=query,max_results=100)
    keywords += len(tweets.data) * [query]
    
    for tweet in tweets.data:
        tweet_list.append(tweet.text)

    

output =[]

for i in tweet_list:
    my_string = i.lower()
    output.append(my_string)

for i in output:
    my_string = i
    doc=nlp(my_string)
    sentiment = doc._.blob.polarity
    
    sentiment = round(sentiment,2)
        
    if sentiment > 0:
        sent_label = "Positive"
    else:
        sent_label = "Negative"

    sent_labels.append(sent_label)
    sent_scores.append(sentiment)

    positive_words = []
    negative_words = []

    for x in doc._.blob.sentiment_assessments.assessments:
        if x[1] > 0:
            positive_words.append(x[0][0])
        elif x[1] < 0:
            negative_words.append(x[0][0])
        else:
            pass

    total_pos.append(', '.join(set(positive_words)))
    total_neg.append(', '.join(set(negative_words)))

In [158]:
df = pd.DataFrame(
    {'Keyword':keywords,'Tweet':output,'Sentiment Score':sent_scores,
    'Sentiment Label': sent_labels,
    'Positive Words':total_pos,'Negative Words':total_neg}
)
df



,Keyword,Tweet,Sentiment Score,Sentiment Label,Positive Words,Negative Words
0,"""University of Chicago"" innovation lang:en -is...",@agrigoi &amp; the development innovation lab ...,-0.25,Negative,,small
1,"""University of Chicago"" innovation lang:en -is...",da&amp;fw inked an mou with development innova...,0.60,Positive,noble,
2,"""University of Chicago"" innovation lang:en -is...",please join us at 4 p.m. on february 9th when ...,0.03,Positive,social,
3,"""University of Chicago"" innovation lang:en -is...",university of chicago launches incubator for a...,0.00,Negative,,
4,UChicago innovation lang:en -is:retweet,@agrigoi &amp; the development innovation lab ...,-0.25,Negative,,small
...,...,...,...,...,...,...
148,UChicago education lang:en -is:retweet,"📣 new podcast! ""episode 6 - space to grow"" on ...",0.17,Positive,new,
149,UChicago education lang:en -is:retweet,"katherine baicker, the emmett dedmon professor...",0.37,Positive,"effective, more",
150,UChicago education lang:en -is:retweet,"""for more than a decade as a chicago public sc...",0.33,Positive,more,
151,UChicago education lang:en -is:retweet,@trans_lingual i think it's fair to include lg...,0.70,Positive,fair,


In [165]:
df.groupby('Keyword').mean()

C:\Users\ambro\AppData\Local\Temp\ipykernel_2888\2278486642.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df.groupby('Keyword').mean()


,Sentiment Score
Keyword,
"""University of Chicago"" innovation lang:en -is:retweet",0.095000
"""University of Chicago"" research lang:en -is:retweet",0.083636
UChicago Therapeutics lang:en -is:retweet,-0.100000
UChicago education lang:en -is:retweet,0.095926
UChicago innovation lang:en -is:retweet,0.016250
UChicago materials lang:en -is:retweet,0.266667
UChicago research lang:en -is:retweet,0.164805
